# Genetic Improvement Demo

### Create the initial population of Genomes

In [1]:
# system imports
import os, sys
from tqdm import tqdm 


# set the project root directory (~/projects/GIMC)
project_root = os.path.expanduser("~/projects/GIMC")

# add project root to sys.path
sys.path.append(project_root)

# project imports
from genetic_improvement.ollamachat import OllamaChat
from genetic_improvement.config import USER_PROMPT, UNIT_TEST_CODE, SYSTEM_PROMPT, NUM_VARIANTS, BSI_CLASSIFICATION
from genetic_improvement.genome import Genome

# CONSTS
POPULATION_SIZE = 8


In [2]:
# get the initial population from the LLM
population = []
for i in tqdm(range(POPULATION_SIZE // NUM_VARIANTS)):
    ollama_chat = OllamaChat(model="qwen3-coder:latest", system_prompt=SYSTEM_PROMPT, temperature=0.3)
    chat_response = ollama_chat.chat(user_text=USER_PROMPT, stream=False)

    # parse the chat response and add it to the population
    variants = OllamaChat.parse_variants(chat_response)
    candidates = OllamaChat.submit_variants(variants, classification=BSI_CLASSIFICATION)
    for candidate in candidates:
        if candidate is not None:
            genome = Genome(candidate_hash=candidate)
            population.append(genome)

 50%|█████     | 1/2 [00:35<00:35, 35.22s/it]

Code submitted successfully:  f2e69282c471196f6783b311e0937f2d25988264637484398c59041375ff17fa
Code submitted successfully:  09830a39657bb5aee6bc0aea85a90b368f5f88c8cbad722fb3a9109c4e643e8e
Code submitted successfully:  27a12a8305ded6ccf6681fe85997618a6e9cb43d30091e66176a970e0259cf9d
Code submitted successfully:  28375f3643ce8d0d7a5ffbc16a608910a17b95a2dca2014fa64a67ee2c06e65e


100%|██████████| 2/2 [01:07<00:00, 33.72s/it]

Code submitted successfully:  5138780550451c6fc12b613409b6e7f8adb6b30d73426ad6888aa50f9c76fdbe
Code submitted successfully:  16e84fe235c900e9056024fbb232accfb2ef6d82423c9b28be91c0c2ffe6947e
Code submitted successfully:  1d0d77e038779e2b9e301ba41979ed9ef9eb9cdaeb82a3011c81ff598ebff037
Code submitted successfully:  e8f56ec227b83ff18ca185afe7fb1aa58bfc53abb5562539579a3bc0a9571225


In [6]:
# get the status of each candidate in the population
for genome in population:
    print(f"Candidate {genome.candidate_hash}")


Candidate f2e69282c471196f6783b311e0937f2d25988264637484398c59041375ff17fa
Candidate 09830a39657bb5aee6bc0aea85a90b368f5f88c8cbad722fb3a9109c4e643e8e
Candidate 27a12a8305ded6ccf6681fe85997618a6e9cb43d30091e66176a970e0259cf9d
Candidate 28375f3643ce8d0d7a5ffbc16a608910a17b95a2dca2014fa64a67ee2c06e65e
Candidate 5138780550451c6fc12b613409b6e7f8adb6b30d73426ad6888aa50f9c76fdbe
Candidate 16e84fe235c900e9056024fbb232accfb2ef6d82423c9b28be91c0c2ffe6947e
Candidate 1d0d77e038779e2b9e301ba41979ed9ef9eb9cdaeb82a3011c81ff598ebff037
Candidate e8f56ec227b83ff18ca185afe7fb1aa58bfc53abb5562539579a3bc0a9571225


In [7]:
# Database setup
import json

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import Sample, Tag, Analysis, Candidate

with open(os.path.join(project_root, "settings.json")) as f:
    settings = json.load(f)
DATABASE_URL = settings['sqlalchemy_database_uri']
engine = create_engine(DATABASE_URL, echo=False)
Session = sessionmaker(bind=engine)

In [12]:
for genome in population:
    print(genome.get_candidate())

Candidate ID: f2e69282c471196f6783b311e0937f2d25988264637484398c59041375ff17fa, Classification: com, Status: 3, F1: 0.25, F2: 0.3333333333333333, F3: 8.109051123028621e-05, Analysis ID: 17946, Error: None
Candidate ID: 09830a39657bb5aee6bc0aea85a90b368f5f88c8cbad722fb3a9109c4e643e8e, Classification: com, Status: 3, F1: 0.0625, F2: 0.0, F3: 0.0, Analysis ID: None, Error: Compilation failed: 5 errors, 0 warnings
Candidate ID: 27a12a8305ded6ccf6681fe85997618a6e9cb43d30091e66176a970e0259cf9d, Classification: com, Status: 3, F1: 0.011363636363636364, F2: 0.0, F3: 0.0, Analysis ID: None, Error: Compilation failed: 29 errors, 0 warnings
Candidate ID: 28375f3643ce8d0d7a5ffbc16a608910a17b95a2dca2014fa64a67ee2c06e65e, Classification: com, Status: 3, F1: 0.14285714285714285, F2: 0.0, F3: 0.0, Analysis ID: None, Error: Compilation failed: 2 errors, 0 warnings
Candidate ID: 5138780550451c6fc12b613409b6e7f8adb6b30d73426ad6888aa50f9c76fdbe, Classification: com, Status: 3, F1: 0.25, F2: 0.333333333333